# Colab Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import lib

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from torchvision import transforms

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from pathlib import Path

import warnings
warnings.filterwarnings(action='ignore') 

# CUDA, Colab path, Config, Random seed 설정 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
CFG = {
    'FPS':30,
    'IMG_SIZE':128,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':42,
    'nfold':5,
}

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [14]:
df = pd.read_csv('/content/drive/MyDrive/hand/open/train.csv')
df['path'] = '/content/drive/MyDrive/hand/open' + df['path'].str[1:]
df

,id,path,label
0,TRAIN_000,/content/drive/MyDrive/hand/open/train/TRAIN_0...,3
1,TRAIN_001,/content/drive/MyDrive/hand/open/train/TRAIN_0...,0
2,TRAIN_002,/content/drive/MyDrive/hand/open/train/TRAIN_0...,1
3,TRAIN_003,/content/drive/MyDrive/hand/open/train/TRAIN_0...,4
4,TRAIN_004,/content/drive/MyDrive/hand/open/train/TRAIN_0...,4
...,...,...,...
605,TRAIN_605,/content/drive/MyDrive/hand/open/train/TRAIN_6...,0
606,TRAIN_606,/content/drive/MyDrive/hand/open/train/TRAIN_6...,2
607,TRAIN_607,/content/drive/MyDrive/hand/open/train/TRAIN_6...,1
608,TRAIN_608,/content/drive/MyDrive/hand/open/train/TRAIN_6...,4


In [15]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

# Code

In [16]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['FPS']):
            _, img = cap.read()
            img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [17]:
train_dataset = CustomDataset(train['path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [18]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=5):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(3, 8, (3, 3, 3)),
            nn.ReLU(),
            nn.BatchNorm3d(8),
            nn.MaxPool3d(2),
            nn.Conv3d(8, 32, (2, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, (2, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.MaxPool3d(2),
            nn.Conv3d(64, 128, (2, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            nn.MaxPool3d((1, 7, 7)),
        )
        self.classifier = nn.Linear(512, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

In [19]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [20]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

# Run

In [41]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.40083] Val Loss : [0.85939] Val F1 : [0.67450]


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.00365] Val Loss : [0.59540] Val F1 : [0.77237]
Epoch 00019: reducing learning rate of group 0 to 9.3750e-06.


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.00353] Val Loss : [0.55981] Val F1 : [0.78741]


# Inference

In [42]:
test = pd.read_csv('/content/drive/MyDrive/hand/open/test.csv')
test['path'] = '/content/drive/MyDrive/hand/open' + test['path'].str[1:]
test

,id,path
0,TEST_000,/content/drive/MyDrive/hand/open/test/TEST_000...
1,TEST_001,/content/drive/MyDrive/hand/open/test/TEST_001...
2,TEST_002,/content/drive/MyDrive/hand/open/test/TEST_002...
3,TEST_003,/content/drive/MyDrive/hand/open/test/TEST_003...
4,TEST_004,/content/drive/MyDrive/hand/open/test/TEST_004...
...,...,...
148,TEST_148,/content/drive/MyDrive/hand/open/test/TEST_148...
149,TEST_149,/content/drive/MyDrive/hand/open/test/TEST_149...
150,TEST_150,/content/drive/MyDrive/hand/open/test/TEST_150...
151,TEST_151,/content/drive/MyDrive/hand/open/test/TEST_151...


In [43]:
test_dataset = CustomDataset(test['path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [44]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [45]:
preds = inference(model, test_loader, device)

  0%|          | 0/39 [00:00<?, ?it/s]

In [46]:
submit = pd.read_csv('/content/drive/MyDrive/hand/open/sample_submission.csv')

In [47]:
submit['label'] = preds
submit.head()

,id,label
0,TEST_000,0
1,TEST_001,3
2,TEST_002,0
3,TEST_003,2
4,TEST_004,4


In [48]:
submit.to_csv('/content/drive/MyDrive/hand/scheduler submit.csv', index=False)